In [1]:
import networkx as nx
import os

# Import given number of files
# NUMBER_OF_GRAPHS_WANTED=-1, to import all files

def import_some(NUMBER_OF_GRAPHS_WANTED,path):
    # Change the directory
    os.chdir(path)
  
    # Read text File

    l = []
  
    def read_file(file_path):
        l.append(nx.read_graphml(file_path))
  
  
    # iterate through all file
    i = -1
    for file in os.listdir():
        i+=1
        if i == NUMBER_OF_GRAPHS_WANTED:
            break
        # Check whether file is in text format or not
        if file.endswith(".graphml"):
            file_path = f"{path}\{file}"
 
            # call read text file function
            read_file(file_path)
    return l

In [2]:
def edge_list(g):
    return list(g.edges)

In [3]:
# Creating the frequency dictionary, l = list of graphs
def frequency_dictionary(l):
    d = {}

    for g in l:
        for edge in edge_list(g):
            if edge in d.keys():
                d[edge] += 1
            else:
                d[edge] = 1
                
    return d

In [4]:
# d = frequency dictionary
# n = number of graphs
# the result is a dictionary, in which pre_percentage_dictionary[i] = list of edges present in i% of graphs, but not in i+1%
import math
def pre_percentage_dictionary(d,n):
    p_p_dict = {i:[] for i in range(101)}
    for key, value in d.items():
        p_p_dict[math.floor(100*value/n)].append(key)
        
    return p_p_dict

In [5]:
# d = frequency dictionary
# n = number of graphs
def percentage_dictionary(d,n):
    percentage = {}
    p_p_dict = pre_percentage_dictionary(d,n)
    
    percentage[100] = p_p_dict[100]
    for i in range(99,-1,-1):
        if percentage[i+1]==[]:
            percentage[i] = p_p_dict[i].copy()
        else:
            percentage[i] = percentage[i+1].copy()
            percentage[i].extend(p_p_dict[i])

    return percentage

In [8]:
def frequency_graphs(path):
    graph_list = import_some(-1, path)
    
    NUMBER_OF_GRAPHS = len(graph_list)
    
    freq_dict = frequency_dictionary(graph_list)
    
    perc_dict = percentage_dictionary(freq_dict, NUMBER_OF_GRAPHS)
    
    os.chdir('..')
    os.mkdir('frequency_graphs_new')
    G0 = nx.Graph()
    G0.add_nodes_from(graph_list[0].nodes(data=True))
#     print(G0.nodes(data=True))
    for node in G0.nodes:
        del G0.nodes[node]['dn_position_x']
        del G0.nodes[node]['dn_position_y']
        del G0.nodes[node]['dn_position_z']
        if int(G0.nodes[node]['dn_correspondence_id']) == int(node):
            del G0.nodes[node]['dn_correspondence_id']
        else:
            print("error")
    for i in range(101):
        G = G0.copy()
        G.add_edges_from(perc_dict[i])
        nx.write_graphml(G, "frequency_graphs_new\\g{}.graphml".format(i))
    
    return perc_dict

In [9]:
path = "..."

pd = frequency_graphs(path)